In [1]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2023-03-09 19:45:30.440989: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 19:45:30.652418: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2.11.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-03-07 20:02:23--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  1.45MB/s    in 0.2s    

2023-03-07 20:02:24 (1.45 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2023-03-07 20:02:24--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[============

In [2]:
df_train = pd.read_table('train-data.tsv', names=['ham_or_spam', 'sms'])
df_val = pd.read_table('valid-data.tsv', names=['ham_or_spam', 'sms'])

In [3]:
df_train.shape, df_val.shape

((4179, 2), (1392, 2))

In [4]:
# check the class distribution
df_train['ham_or_spam'].value_counts()

ham     3619
spam     560
Name: ham_or_spam, dtype: int64

In [5]:
# check the class distribution in the validation set
df_val['ham_or_spam'].value_counts()

ham     1205
spam     187
Name: ham_or_spam, dtype: int64

In [7]:
# view 5 random samples from the training set
df_train.sample(5)

,ham_or_spam,sms
1408,ham,anyway seriously hit me up when you're back be...
3072,spam,we tried to contact you re our offer of new vi...
3607,ham,in that case i guess i'll see you at campus lodge
2136,ham,take some small dose tablet for fever
2421,ham,"and how you will do that, princess? :)"


In [8]:
# create a label column for the model
df_train['label'] = df_train['ham_or_spam'].map({'ham': 0, 'spam': 1})
df_val['label'] = df_val['ham_or_spam'].map({'ham': 0, 'spam': 1})

In [9]:
# set up class weights to account for the small number of 'spam' samples
neg = (df_train['label'] == 0).sum()
pos = (df_train['label'] == 1).sum()
neg, pos

(3619, 560)

In [10]:
counts = df_train['label'].value_counts()
neg = counts[0]
pos = counts[1]

weight_for_0 = (1 / neg) * (len(df_train) / 2.0)
weight_for_1 = (1 / pos) * (len(df_train) / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)

{0: 0.5773694390715667, 1: 3.7312499999999997}


In [13]:
df_train.head()

,ham_or_spam,sms,label
0,ham,ahhhh...just woken up!had a bad dream about u ...,0
1,ham,you can never do nothing,0
2,ham,"now u sound like manky scouse boy steve,like! ...",0
3,ham,mum say we wan to go then go... then she can s...,0
4,ham,never y lei... i v lazy... got wat? dat day ü ...,0


In [11]:
# set up train and validation data in numpy arrays
train_texts = df_train['sms'].to_numpy()
train_labels = df_train['label'].to_numpy()
val_texts = df_val['sms'].to_numpy()
val_labels = df_val['label'].to_numpy()

In [46]:
pd.set_option('display.max_colwidth', 200)
# inspect a sample of the training data
df_sample = df_train.sample(10)
# add a column with the number of words in each text
df_sample['tokens'] = df_sample['sms'].apply(lambda x: len(x.split()))
df_sample

,ham_or_spam,sms,label,tokens
1743,ham,i was about to do it when i texted. i finished a long time ago and showered and er'ything!,0,19
620,ham,"haha awesome, i might need to take you up on that, what you doin tonight?",0,15
1252,ham,i'm leaving my house now.,0,5
3165,ham,how come guoyang go n tell her? then u told her?,0,11
2191,ham,hello which the site to download songs its urgent pls,0,10
651,ham,o we cant see if we can join denis and mina? or does denis want alone time,0,17
2997,ham,hi its kate how is your evening? i hope i can see you tomorrow for a bit but i have to bloody babyjontet! txt back if u can. :) xxx,0,30
2104,ham,u wan 2 haf lunch i'm in da canteen now.,0,10
1898,ham,"ya, told..she was asking wats matter?",0,6
2878,ham,i not free today i haf 2 pick my parents up tonite...,0,12


## Set up text processing layers

In [ ]:
# find the max number of tokens in train_texts
max_length = max([len(i.split()) for i in train_texts])
max_length

171

In [47]:
# find the median number of tokens in train_texts
median_length = np.median([len(i.split()) for i in train_texts])
median_length

12.0

In [48]:
# find the average number of tokens in train_texts
avg_length = np.mean([len(i.split()) for i in train_texts])
avg_length

15.604450825556354

In [49]:
from tensorflow.keras.layers import TextVectorization

# max number of words to use in our vocab
max_vocab_length = 10000
# max tokens in each text
output_sequence_length = 30

# create a text vectorizer layer
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode='int',
                                    output_sequence_length=output_sequence_length)
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_texts)

In [13]:
# test out the text vectorizer with a sample text
sample_text = 'I love machine learning'
text_vectorizer([sample_text])

<tf.Tensor: shape=(1, 16), dtype=int64, numpy=array([[ 3, 69,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])>

In [14]:
# Create an embedding layer
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,
                                      output_dim=128, # output shape,
                                      input_length=max_length)

In [29]:
# Get a random sentence from the training set
import random
random_sentence = random.choice(train_texts)
print(random_sentence)

embedding(text_vectorizer([random_sentence]))

hi dear we saw dear. we both are happy. where you my battery is low


<tf.Tensor: shape=(1, 16, 128), dtype=float32, numpy=
array([[[-0.02435087,  0.03975994, -0.00540585, ..., -0.0358328 ,
         -0.0108956 ,  0.03641697],
        [ 0.01703156, -0.0277077 , -0.02375525, ..., -0.04986947,
          0.02598711,  0.04096658],
        [ 0.0400219 ,  0.04994031,  0.04106606, ...,  0.02158229,
         -0.03520878,  0.01922883],
        ...,
        [ 0.03981615, -0.02227314, -0.04821194, ...,  0.04713095,
          0.02334302, -0.03417813],
        [ 0.02281949, -0.01116902, -0.02656424, ...,  0.0171528 ,
          0.02070067, -0.03872194],
        [ 0.00640814, -0.04977547, -0.0315617 , ..., -0.02413393,
          0.00196383,  0.00353675]]], dtype=float32)>

## Model 1 - simple Dense model

In [77]:
# build the model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of those numbers
x = layers.GlobalAveragePooling1D()(x) # condense the embedding down to one vector
outputs = layers.Dense(1, activation='sigmoid')(x) # create an output layer
model_1 = tf.keras.Model(inputs, outputs, name='model_1')


In [59]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       multiple                  1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-tra

In [78]:
# Compile the model
model_1.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]) 

In [79]:
# Fit the model
history_1 = model_1.fit(train_texts,
            train_labels,
            epochs=10,
            validation_data=(val_texts, val_labels),
            class_weight=class_weight)

Epoch 1/20
131/131 [==============================] - 1s 7ms/step - loss: 0.4179 - accuracy: 0.9584 - precision_7: 0.8755 - recall_7: 0.8036 - val_loss: 0.2057 - val_accuracy: 0.9806 - val_precision_7: 0.9819 - val_recall_7: 0.8717
Epoch 2/20
131/131 [==============================] - 1s 6ms/step - loss: 0.1646 - accuracy: 0.9933 - precision_7: 1.0000 - recall_7: 0.9500 - val_loss: 0.1280 - val_accuracy: 0.9813 - val_precision_7: 0.9708 - val_recall_7: 0.8877
Epoch 3/20
131/131 [==============================] - 1s 6ms/step - loss: 0.0961 - accuracy: 0.9955 - precision_7: 0.9963 - recall_7: 0.9696 - val_loss: 0.1007 - val_accuracy: 0.9835 - val_precision_7: 0.9659 - val_recall_7: 0.9091
Epoch 4/20
131/131 [==============================] - 1s 6ms/step - loss: 0.0669 - accuracy: 0.9976 - precision_7: 0.9964 - recall_7: 0.9857 - val_loss: 0.0841 - val_accuracy: 0.9864 - val_precision_7: 0.9667 - val_recall_7: 0.9305
Epoch 5/20
131/131 [==============================] - 1s 6ms/step - loss

In [62]:
# evaluate
model_1.evaluate(val_texts, val_labels)

44/44 [==============================] - 0s 1ms/step - loss: 0.0608 - accuracy: 0.9878 - precision_4: 0.9620 - recall_4: 0.9465


[0.06076987460255623,
 0.9877873659133911,
 0.9619565010070801,
 0.9465240836143494]

In [65]:
test_predictions(model_1)

1/1 [==============================] - 0s 27ms/step
You passed the challenge. Great job!


## Model 2 - LSTM

In [66]:
# build the model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, outputs, name='model_2')

In [67]:
# compile the model
model_2.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [68]:
# fit the model
history_2 = model_2.fit(train_texts,
                        train_labels,
                        epochs=10,
                        validation_data=(val_texts, val_labels),
                        class_weight=class_weight)

Epoch 1/10
131/131 [==============================] - 9s 15ms/step - loss: 0.0701 - accuracy: 0.9868 - precision_5: 0.9287 - recall_5: 0.9768 - val_loss: 0.1218 - val_accuracy: 0.9784 - val_precision_5: 0.9026 - val_recall_5: 0.9412
Epoch 2/10
131/131 [==============================] - 2s 13ms/step - loss: 0.0068 - accuracy: 0.9988 - precision_5: 0.9912 - recall_5: 1.0000 - val_loss: 0.0848 - val_accuracy: 0.9842 - val_precision_5: 0.9412 - val_recall_5: 0.9412
Epoch 3/10
131/131 [==============================] - 2s 13ms/step - loss: 0.0045 - accuracy: 0.9990 - precision_5: 0.9929 - recall_5: 1.0000 - val_loss: 0.0888 - val_accuracy: 0.9842 - val_precision_5: 0.9459 - val_recall_5: 0.9358
Epoch 4/10
131/131 [==============================] - 2s 13ms/step - loss: 0.0035 - accuracy: 0.9990 - precision_5: 0.9929 - recall_5: 1.0000 - val_loss: 0.0782 - val_accuracy: 0.9856 - val_precision_5: 0.9665 - val_recall_5: 0.9251
Epoch 5/10
131/131 [==============================] - 2s 12ms/step -

In [69]:
# evaluate the model
model_2.evaluate(val_texts, val_labels)

44/44 [==============================] - 0s 3ms/step - loss: 0.1077 - accuracy: 0.9856 - precision_5: 0.9563 - recall_5: 0.9358


[0.10771406441926956,
 0.9856321811676025,
 0.9562841653823853,
 0.9358288645744324]

In [70]:
test_predictions(model_2)

1/1 [==============================] - 0s 26ms/step
You passed the challenge. Great job!


## Model 3: Multi-layer LSTM

In [71]:
# build the model
inputs = layers.Input(shape=(1,), dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs, name='model_3')

In [72]:
model_3.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 30)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       multiple                  1280000   
                                                                 
 lstm_6 (LSTM)               (None, 30, 64)            49408     
                                                                 
 lstm_7 (LSTM)               (None, 64)                33024     
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                           

In [73]:
# compile the model
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

In [74]:
# fit the model
history_3 = model_3.fit(train_texts,
                        train_labels,
                        epochs=5,
                        validation_data=(val_texts, val_labels),
                        class_weight=class_weight)

Epoch 1/5
131/131 [==============================] - 5s 23ms/step - loss: 0.0561 - accuracy: 0.9916 - precision_6: 0.9472 - recall_6: 0.9929 - val_loss: 0.1281 - val_accuracy: 0.9784 - val_precision_6: 0.9067 - val_recall_6: 0.9358
Epoch 2/5
131/131 [==============================] - 2s 19ms/step - loss: 0.0123 - accuracy: 0.9969 - precision_6: 0.9790 - recall_6: 0.9982 - val_loss: 0.0650 - val_accuracy: 0.9864 - val_precision_6: 0.9565 - val_recall_6: 0.9412
Epoch 3/5
131/131 [==============================] - 2s 19ms/step - loss: 0.0088 - accuracy: 0.9993 - precision_6: 0.9964 - recall_6: 0.9982 - val_loss: 0.1203 - val_accuracy: 0.9813 - val_precision_6: 0.9215 - val_recall_6: 0.9412
Epoch 4/5
131/131 [==============================] - 3s 21ms/step - loss: 0.0038 - accuracy: 0.9993 - precision_6: 0.9947 - recall_6: 1.0000 - val_loss: 0.0992 - val_accuracy: 0.9842 - val_precision_6: 0.9558 - val_recall_6: 0.9251
Epoch 5/5
131/131 [==============================] - 4s 28ms/step - loss

In [75]:
model_3.evaluate(val_texts, val_labels)

44/44 [==============================] - 0s 6ms/step - loss: 0.1048 - accuracy: 0.9799 - precision_6: 0.8995 - recall_6: 0.9572


[0.10481208562850952,
 0.9798850417137146,
 0.8994975090026855,
 0.9572192430496216]

In [76]:
test_predictions(model_3)

1/1 [==============================] - 0s 28ms/step
You passed the challenge. Great job!


In [27]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text, model=model_1):
  prediction = round(model.predict([pred_text])[0][0])

  if prediction == 0:
    return [prediction, 'ham']
  else:
    return [prediction, 'spam']

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 81ms/step
[0, 'ham']


In [64]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions(model):
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg, model)
    # print(f'{prediction[1]}: {msg}')
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

# test_predictions()
